In [ ]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns

DIR = '/mnt/ml-team/minerva/open-solutions/home-credit'
description = pd.read_csv(os.path.join(DIR,'data/HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'files/unzipped_data/application_train.csv'))
installments = pd.read_csv(os.path.join(DIR, 'files/unzipped_data/installments_payments.csv'))

In [ ]:
installments.head()

# Preprocessing
## Solution 3

# Feature Engineering
## Solution 3

## Aggregations

In [ ]:
INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES = []
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_INSTALMENT',
                   'AMT_PAYMENT',
                   'DAYS_ENTRY_PAYMENT',
                   'DAYS_INSTALMENT',
                   'NUM_INSTALMENT_NUMBER',
                   'NUM_INSTALMENT_VERSION'
                   ]:
        INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES.append((select, agg))
INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES)]

In [ ]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES):
    group_object = installments.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        application = application.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [ ]:
application.head()

In [ ]:
application_agg = application[groupby_aggregate_names + ['TARGET']]
application_agg_corr = abs(application_agg.corr())

In [ ]:
application_agg_corr.sort_values('TARGET', ascending=False)['TARGET']

# Solution 4

In [ ]:
positive_ID = application[application['TARGET']==1]['SK_ID_CURR'].tolist()
positive_ID[:4]

In [ ]:
value_counts = installments[installments['SK_ID_CURR'].isin(positive_ID)]['SK_ID_CURR'].value_counts()

In [ ]:
value_counts.head()

In [ ]:
sns.distplot(value_counts)

In [ ]:
installments_one = installments[installments['SK_ID_CURR']==328162]

In [ ]:
installments_one.sort_values(['DAYS_INSTALMENT'],ascending=False).head(10)

In [ ]:
installments_ = installments.sample(500)
installments_['instalment_paid_late_in_days'] = installments_['DAYS_ENTRY_PAYMENT'] - installments_['DAYS_INSTALMENT'] 
installments_['instalment_paid_late'] = (installments_['instalment_paid_late_in_days'] > 0).astype(int)
installments_['instalment_paid_over_amount'] = installments_['AMT_PAYMENT'] - installments_['AMT_INSTALMENT']
installments_['instalment_paid_over'] = (installments_['instalment_paid_over_amount'] > 0).astype(int)

In [ ]:
def add_features(feature_name, aggs, features, feature_names, groupby):
    feature_names.extend(['{}_{}'.format(feature_name,agg) for agg in aggs])

    for agg in aggs:
        g = groupby[feature_name].agg(agg).reset_index().rename(index=str, 
                                                         columns={feature_name:'{}_{}'.format(feature_name,agg)})
        features = features.merge(g, on='SK_ID_CURR', how='left')
    return features, feature_names

In [ ]:
features = pd.DataFrame({'SK_ID_CURR':installments_['SK_ID_CURR'].unique()})
groupby = installments_.groupby(['SK_ID_CURR'])

feature_names = []

features, feature_names = add_features('instalment_paid_late_in_days', ['sum','mean','max','min','std'],
                                     features, feature_names, groupby)

features, feature_names = add_features('instalment_paid_late', ['sum','mean'],
                                     features, feature_names, groupby)

features, feature_names = add_features('instalment_paid_over_amount', ['sum','mean','max','min','std'],
                                     features, feature_names, groupby)

features, feature_names = add_features('instalment_paid_over', ['sum','mean'],
                                     features, feature_names, groupby)
    
display(features.head())

In [ ]:
def last_installment_features(gr):
    gr_ = gr.copy()
    gr_.sort_values(['DAYS_INSTALMENT'],ascending=False, inplace=True)
    last_instalment_id = gr_['SK_ID_PREV'].iloc[0]
    gr_ = gr_[gr_['SK_ID_PREV']==last_instalment_id]
    features = {}
    
    feature_name = 'instalment_paid_late_in_days'
    features['last_{}_sum'.format(feature_name)] = gr_[feature_name].sum()
    features['last_{}_mean'.format(feature_name)] = gr_[feature_name].mean()
    features['last_{}_max'.format(feature_name)] = gr_[feature_name].max()
    features['last_{}_min'.format(feature_name)] = gr_[feature_name].min()
    features['last_{}_std'.format(feature_name)] = gr_[feature_name].std()

    feature_name = 'instalment_paid_late'
    features['last_{}_count'.format(feature_name)] = gr_[feature_name].count()
    features['last_{}_mean'.format(feature_name)] = gr_[feature_name].mean()
    
    feature_name = 'instalment_paid_over_amount'
    features['last_{}_sum'.format(feature_name)] = gr_[feature_name].sum()
    features['last_{}_mean'.format(feature_name)] = gr_[feature_name].mean()
    features['last_{}_max'.format(feature_name)] = gr_[feature_name].max()
    features['last_{}_min'.format(feature_name)] = gr_[feature_name].min()
    features['last_{}_std'.format(feature_name)] = gr_[feature_name].std()

    feature_name = 'instalment_paid_over'
    features['last_{}_count'.format(feature_name)] = gr_[feature_name].count()
    features['last_{}_mean'.format(feature_name)] = gr_[feature_name].mean()

    return pd.Series(features)

In [ ]:
import multiprocessing as mp

from tqdm import tqdm_notebook as tqdm
import numpy as np


def chunk_groups(groupby_object, chunk_size):
    n_groups = groupby_object.ngroups
    group_chunk, index_chunk = [],[]
    for i, (index, df) in enumerate(groupby_object):
        group_chunk.append(df)
        index_chunk.append(index)

        if (i + 1) % chunk_size == 0 or i + 1 == n_groups:
            group_chunk_, index_chunk_ = group_chunk.copy(), index_chunk.copy()
            group_chunk, index_chunk = [],[]
            yield index_chunk_, group_chunk_


def parallel_apply(groups, func, index_name='Index', num_workers=1, chunk_size=1000):
    n_chunks = np.ceil(1.0 * groups.ngroups / chunk_size)
    indeces, features  = [],[]
    for index_chunk, groups_chunk in tqdm(chunk_groups(groups, chunk_size), total=n_chunks):
        with mp.pool.Pool(num_workers) as executor:
            features_chunk = executor.map(func, groups_chunk)
        features.extend(features_chunk)
        indeces.extend(index_chunk)

    features = pd.DataFrame(features)
    features.index = indeces
    features.index.name = index_name
    return features

In [ ]:
g = groupby.apply(last_installment_features).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')

display(features.head())

In [ ]:
g = parallel_apply(groupby, last_installment_features, index_name='SK_ID_CURR',
                   num_workers=16, chunk_size=10000).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')

display(features.head())

In [ ]:
X = application.merge(features, on='SK_ID_CURR',how='left')
X = X[features.columns.drop('SK_ID_CURR').tolist()+['TARGET']]
X.head()

In [ ]:
X_corr = abs(X.corr())
X_corr.sort_values('TARGET', ascending=False)['TARGET']